In [1]:
%%script false --no-raise-error
import json
from google.colab import userdata
from google.oauth2 import service_account
from google.cloud.bigquery import magics

credentials_json = userdata.get('BIGQUERY_CREDENTIALS')
credentials = service_account.Credentials.from_service_account_info(json.loads(credentials_json))
magics.context.credentials = credentials

Couldn't find program: 'false'


In [2]:
from google.cloud import bigquery
from google.cloud.bigquery import magics
%load_ext bigquery_magics

data_set = "testing_set"
project_name = "emerald-entity-468916-f9"

job_config = bigquery.QueryJobConfig(default_dataset = f"{project_name}.{data_set}")
client = bigquery.Client(project = project_name, default_query_job_config = job_config, credentials = globals().get('credentials', None))
magics.context.default_query_job_config = job_config
magics.context.project = project_name

### Phase 7 - Clustering
**It takes the raw data gathered for each interesting us trait and clusters them together into consistent groups.**
**The cluster sizes are chosen arbitrary and can be changed in last three cells (first line in each cell)**
**Running clustering again, will regenerate clusters without damaging the core data.**

In [ ]:
%%bigquery

UPDATE characters SET social_class_cluster_id = NULL, wealth_cluster_id = NULL, values_cluster_id = NULL WHERE 1 = 1;
DELETE FROM clusters WHERE 1 = 1;
DELETE FROM character_cluster_details WHERE 1 = 1;

Query is running:   0%|          |

""


In [ ]:
%%bigquery --params {"clusters": 10, "trait_type": "values", "trait_desc": 'values: "Core principles, moral compass, priorities (can include both positive and negative values)"'}

create or replace table tmp_copied_traits as SELECT c.book_id, c.id, c.values as traits from characters c WHERE c.values is not null;
CALL phase7_split_traits(@trait_desc);
CALL phase7_identify_clusters(@clusters);
CALL phase7_name_clusters(@trait_type, @trait_desc);
CALL phase7_assign_clusters(@trait_type);
UPDATE characters c SET values_cluster_id = ct.cluster_id FROM tmp_combined_traits ct WHERE c.book_id = ct.book_id AND c.id = ct.id AND ct.cluster_type = @trait_type;

Executing query with job ID: 5c8823ef-232e-4119-8754-936bc852d0d2
Query executing: 83.05s

In [ ]:
%%bigquery --params {"clusters": 7, "trait_type": "wealth", "trait_desc": 'wealth: "Economic position, assets, property, financial struggles or abundance with information how wealth/income is obtained (inheritance, labor, trade, crime, patronage, etc.)"'}

create or replace table tmp_copied_traits as SELECT c.book_id, c.id, c.wealth as traits from characters c WHERE c.wealth is not null;
CALL phase7_split_traits(@trait_desc);
CALL phase7_identify_clusters(@clusters);
CALL phase7_name_clusters(@trait_type, @trait_desc);
CALL phase7_assign_clusters(@trait_type);
UPDATE characters c SET wealth_cluster_id = ct.cluster_id FROM tmp_combined_traits ct WHERE c.book_id = ct.book_id AND c.id = ct.id AND ct.cluster_type = @trait_type;

In [ ]:
%%bigquery --params {"clusters": 7, "trait_type": "social_class", "trait_desc": 'social_class: "Economic and social standing (e.g., nobility, working class, merchant class)"'}

create or replace table tmp_copied_traits as SELECT c.book_id, c.id, c.social_class as traits from characters c WHERE c.social_class is not null;
CALL phase7_split_traits(@trait_desc);
CALL phase7_identify_clusters(@clusters);
CALL phase7_name_clusters(@trait_type, @trait_desc);
CALL phase7_assign_clusters(@trait_type);
UPDATE characters c SET social_class_cluster_id = ct.cluster_id FROM tmp_combined_traits ct WHERE c.book_id = ct.book_id AND c.id = ct.id AND ct.cluster_type = @trait_type;